In [332]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [333]:
df = pd.read_csv('Data/player_projections.csv')

In [334]:
for team in df['Team'].unique():
    print(df[df['Team'] == team])

                           Team            Player  Week  Rank  \
0     Hot Chubb Time  Machine 2  Justin Jefferson     1     5   
1     Hot Chubb Time  Machine 2  Justin Jefferson     2     6   
2     Hot Chubb Time  Machine 2  Justin Jefferson     3    30   
3     Hot Chubb Time  Machine 2  Justin Jefferson     4    13   
4     Hot Chubb Time  Machine 2  Justin Jefferson     5    16   
...                         ...               ...   ...   ...   
2328  Hot Chubb Time  Machine 2      Greg Dulcich    12    13   
2345  Hot Chubb Time  Machine 2     Isaiah Likely     9     2   
2370  Hot Chubb Time  Machine 2  Christian Watson    11    32   
2371  Hot Chubb Time  Machine 2  Christian Watson    12     6   
2375  Hot Chubb Time  Machine 2   Taylor Heinicke    12    23   

      Projected_Points  Actual_Points Position  Total_Expected  Total_Actual  \
0                17.87          39.40       WR          287.80        327.06   
1                19.73          10.80       WR          287

In [335]:
df.groupby('Week', as_index = False)

for team in df['Team'].unique():
    for position in df['Position'].unique():
        for week in df['Week'].unique():
            print(df[(df['Team'] == team) & (df['Position'] == position) & (df['Week'] == week)].sort_values('Actual_Points', ascending=False).reset_index())

   index                       Team            Player  Week  Rank  \
0      0  Hot Chubb Time  Machine 2  Justin Jefferson     1     5   
1     48  Hot Chubb Time  Machine 2  Courtland Sutton     1     3   
2    127  Hot Chubb Time  Machine 2       Romeo Doubs     1    31   
3     84  Hot Chubb Time  Machine 2     Brandon Aiyuk     1    16   
4     60  Hot Chubb Time  Machine 2    Darnell Mooney     1    20   
5     99  Hot Chubb Time  Machine 2    George Pickens     1    10   

   Projected_Points  Actual_Points Position  Total_Expected  Total_Actual  \
0             17.87           39.4       WR          287.80        327.06   
1             12.82           11.2       WR          210.42        127.30   
2              9.79            8.8       WR          120.19         90.00   
3             10.72            6.7       WR          184.92        178.10   
4             11.62            1.8       WR          212.18        101.50   
5              8.57            1.3       WR          1

In [336]:
df[(df['Team'] == '3 PP Wine') & (df['Position'] == 'RB') & (df['Week'] == 10)].sort_values('Actual_Points', ascending=False)


,Team,Player,Week,Rank,Projected_Points,Actual_Points,Position,Total_Expected,Total_Actual,Point-Residuals
523,3 PP Wine,James Conner,10,14,12.34,23.6,RB,240.04,161.20,11.26
511,3 PP Wine,Christian McCaffrey,10,27,22.59,17.7,RB,309.94,294.36,-4.89
2180,3 PP Wine,Eno Benjamin,10,10,14.37,9.5,RB,64.37,86.40,-4.87
579,3 PP Wine,Miles Sanders,10,5,12.68,5.4,RB,184.65,201.20,-7.28
2141,3 PP Wine,Gus Edwards,10,13,9.99,1.2,RB,84.01,47.60,-8.79


In [337]:
# # Got each of the top players for each position for each team!
# # Wait i need 2 rbs and 2 wrs
# max_df = pd.DataFrame()
# for week in df['Week'].unique():
#     for team in df['Team'].unique():
#         second_max_points = 0
#         second_max_row = pd.Series()
#         for pos in df['Position'].unique():
#             #Get best player for each position
#             if df[(df['Team'] == team) & (df['Position'] == pos) & (df['Week'] == week)]['Actual_Points'].size: # if the player has points
#                 index_max = df[(df['Team'] == team) & (df['Position'] == pos) & (df['Week'] == week)]['Actual_Points'].nlargest(2)

#                 #Find the flex player
#                 if df[(df['Team'] == team) & (df['Position'] == pos) & (df['Week'] == week)]['Actual_Points'].nlargest(3).size > 1:
#                     second_max = df[(df['Team'] == team) & (df['Position'] == pos) & (df['Week'] == week)].nlargest(2, 'Actual_Points')
#                     print(second_max.iloc[1]['Actual_Points'])
#                     if pos != 'QB':
#                         if second_max_points < second_max.iloc[1]['Actual_Points']:
#                             second_max_points = second_max.iloc[1]['Actual_Points']
#                             second_max_row = second_max.iloc[1]
                          
#                 max_df = pd.concat([max_df, pd.DataFrame(index_max).T])
#         max_df = pd.concat([max_df, pd.DataFrame(second_max_row).T])
#         print(second_max_row.to_frame)

In [391]:
def add_player_to_df(row, df, dic, max_df):
    
    array_split = []
    if dic:
        for pos in df['Position'].unique():
            if df[df['Position'] == pos].size:
                # if pos == 'DST':
                #     continue
                arr = []

                arr = dic[pos].split(',')
                if pos == 'RB' or pos == 'WR' or pos == 'TE' or pos == 'K':
                    array_split = arr[:2]
                else:
                    array_split = arr[:1]
                for player in array_split:
                    # print(df[(df['Player'] == player) & (df['Week'] == week)])
                    max_df = max_df.append(df[(df['Player'] == player) & (df['Week'] == week)])
                    pd.concat([max_df, df[(df['Player'] == player) & (df['Week'] == week)]])
    return max_df


In [395]:
df = pd.read_csv('Data/player_projections.csv')
max_df = pd.DataFrame()

df[df['Position'] == 'D/ST'] = 'DST'
for week in df['Week'].unique():
    for team in df['Team'].unique():
        dic = {}
        temp_df = df[(df['Team'] == team) & (df['Week'] == week)].sort_values('Actual_Points', ascending=False)

        #Put positions and players into a dic
        for index, row in temp_df.iterrows():
            if row['Position'] not in dic:
                dic[row['Position']] = row['Player']
            else:
                dic[row['Position']] = dic[row['Position']] + "," + row['Player']
        # Put Get top 2 for each position
        # max_df = add_player_to_df(row, df, dic, max_df)
        print(dic)

{'WR': 'Justin Jefferson,Courtland Sutton,Romeo Doubs,Brandon Aiyuk,Darnell Mooney,George Pickens', 'RB': 'Nick Chubb,Darrell Henderson Jr.,Kenneth Gainwell,Aaron Jones,Chase Edmonds,Travis Etienne Jr.', 'K': 'Ryan Succop', 'QB': 'Trey Lance', 'TE': 'Cole Kmet'}
{'WR': 'A.J. Brown,Gabe Davis,Jaylen Waddle,Elijah Moore,Marvin Jones Jr.', 'RB': 'Joe Mixon,James Robinson,Leonard Fournette,Damien Harris,J.D. McKissic,Tyler Allgeier', 'QB': 'Joe Burrow', 'TE': 'Zach Ertz,Irv Smith Jr.', 'K': 'Daniel Carlson'}
{'RB': 'Saquon Barkley,Nyheim Hines,Najee Harris,Elijah Mitchell,Mike Davis', 'WR': 'Julio Jones,Mecole Hardman,Chase Claypool,Adam Thielen,Tee Higgins,Mike Williams,Allen Robinson II,DeVante Parker,Michael Gallup', 'K': 'Rodrigo Blankenship', 'QB': 'Aaron Rodgers', 'TE': 'Dawson Knox'}
{'WR': 'Davante Adams,DJ Chark,Diontae Johnson,Marquez Valdes-Scantling,Robert Woods', 'RB': 'Cordarrelle Patterson,Breece Hall,Alvin Kamara,Rhamondre Stevenson,Damien Williams', 'QB': 'Kyler Murray,Kir

In [350]:
# for pos in df['Position'].unique():
#         if df[df['Position'] == pos].size:
#             if pos == 'D/ST':
#                 continue
#             arr = dic[pos].split(',')
#             for player in arr[:2]:
#                 pd.concat([max_df, df[(df['Player'] == player)]])
            

In [351]:
max_df.to_csv('output.csv')

In [352]:
max_df

""
